# Polarizability tensor from a workflow class

The goal of this notebook is to present a first implementation of a workflow class in BigDFT. The goal of such classes is to allow the treatment of a given set of calculations and their post-processing whatever the BigDFT calculator used.

This notebook deals with the calculation of the polarizability tensor. It shows that the same polarizability tensor is obtained by using two types of calculators.

## Initialization

### Import the necessary classes

The class allowing to find the polarizability tensor for a given structure of a molecule is `PolTensorWorkflow`. In order to be initialized, it requires a calculator as first argument, hence the import of two calculators.

In [1]:
from poltensorworkflow import PolTensorWorkflow
from BigDFT.Calculators import GIBinding, SystemCalculator

### Define initial variables

The basic input file is defined as a string. It must contain the information about the initial position. For the moment, only string input files can be used. (A workaround of these limitations would be to define classes for the input and posinp files; a first fraft of such classes can be found in `inputfiles.py`; another is also to enforce similar APIs for the BigDFT calculators, say by means of abstract base classes).

Note that the BigDFT variables `gnrm_cv` and `hgrids` are high in this example: it allows for a shorter computation time.

In [2]:
input_base = """\
dft:
  hgrids: 0.45
  ixc: LDA
  gnrm_cv: 1.e-2
posinp:
  positions:                                                                   
  - O: [-2.868578405923472E-007, 1.762479051592436E-015, 0.7354311347007751]   
  - H: [1.463926076889038, 2.55351295663786E-015, -0.367715448141098]          
  - H: [-1.463925838470459, 2.1094237467877974E-015, -0.3677156567573547]      
  properties:                                                                  
    format: xyz"""

To compute the polarizability tensor, one must compute the variation of the dipole moment of the molecule along one direction with respect to an infinitesimal variation of the electric field amplitude in one direction. To do that by means of second order finite difference, six DFT calculations must be performed, that is two per space coordinate: one with a positive electric field amplitude, the other with a negative one, this being repeated for an electric field along the $x$, $y$ and $z$ axis. 

This means that the electric field amplitudes along each direction must be defined. Note that if this amplitude in one direction is `None` or 0, then no computation will be performed. This is useful when there are surface boundary conditions (at the cost of neglecting most terms in the polarizability tensor).

In [3]:
ef_amplitudes = [None, 1.e-3, 0]

## Polarizability tensor with the GIBinding class

The polarizability tensor can be computed thanks to the GIBinding calculator, as presented below. One must simply initialize the workflow with a calculator, an input file as a string (default values for the electric field amplitudes could be used instead of the ones above; see last example of this notebook). Given that the electric field amplitudes are non-zero only along the $y$ axis, only two calculations will be performed.

In [4]:
calc1 = GIBinding()
pt1 = PolTensorWorkflow(calc1, input_base, ef_amplitudes=ef_amplitudes)
pt1.run()

No negative electric field applied along x!
No positive electric field applied along z!
No positive electric field applied along x!
No negative electric field applied along z!
Positive electric field applied along y!
Negative electric field applied along y!


And... that's it! The polarizability tensor can then be returned by the `pol_tensor` attribute:

In [5]:
print(pt1.pol_tensor)

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -1.60000000e-04   1.02307500e+01   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]


Note that the only non-zero terms are the ones concerning the electric field along the $y$ axis.

In [6]:
# The GIBinding calculator has to be deleted properly for the other parts of this notebook to run
del(calc1)
del(pt1)

## Polarizability tensor with the SystemCalculator class

Another calculator can be used, it won't change the results:

In [7]:
calc2 = SystemCalculator(omp=4)
pt2 = PolTensorWorkflow(calc2, input_base, ef_amplitudes=ef_amplitudes)
pt2.run()

Initialize a Calculator with OMP_NUM_THREADS=4 and command /bigdft/bin/bigdft
No negative electric field applied along x!
No positive electric field applied along z!
No positive electric field applied along x!
No negative electric field applied along z!
Executing command: /bigdft/bin/bigdft -n EF_along_y+
Positive electric field applied along y!
Executing command: /bigdft/bin/bigdft -n EF_along_y-
Negative electric field applied along y!


In [8]:
print(pt2.pol_tensor)

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -1.60000000e-04   1.02307500e+01   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]


## Basic use case

You could just use the default electric field amplitudes to get the whole polarizability tensor, without having to worry about the electric field amplitude:

In [9]:
pt3 = PolTensorWorkflow(calc2, input_base)
pt3.run()

Executing command: /bigdft/bin/bigdft -n EF_along_x-
Negative electric field applied along x!
Executing command: /bigdft/bin/bigdft -n EF_along_z+
Positive electric field applied along z!
Executing command: /bigdft/bin/bigdft -n EF_along_x+
Positive electric field applied along x!
Executing command: /bigdft/bin/bigdft -n EF_along_z-
Negative electric field applied along z!
Executing command: /bigdft/bin/bigdft -n EF_along_y+
Positive electric field applied along y!
Executing command: /bigdft/bin/bigdft -n EF_along_y-
Negative electric field applied along y!


The polarizability tensor is then completely defined:

In [10]:
print(pt3.pol_tensor)

[[  1.08742500e+01  -5.00000000e-05   5.00000000e-02]
 [ -1.50000000e-04   1.02299000e+01   5.00000000e-02]
 [  3.00000000e-04   3.70000000e-03   1.05500000e+01]]


Note that these values are not converged: both BigDFT parameters `hgrids` and `gnrm_cv` should be decreased to achieve convergence.